In [1]:
import os 
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename


import cv2
from keras.models import load_model 
from keras.backend import set_session
from skimage.transform import resize 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import numpy as np
import pydicom as dicom


from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPooling2D , Flatten , Dropout , BatchNormalization
from keras.callbacks import ReduceLROnPlateau

In [2]:
def create_model():
    # Creating a model taking input as 256*256 image
   
    model = Sequential()
    model.add(Conv2D(256, (10, 10), activation='relu', input_shape=(256,256,1)))
    model.add(MaxPooling2D((10, 10)))
    model.add(Conv2D(256, (10, 10), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics = 'accuracy')
    return model

In [ ]:

model = create_model()
model.load_weights('simple_model.h5')

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def main_page():
    if request.method == 'POST':
        file = request.files['file']
        file.save('temp.dcm')
        return redirect(url_for('prediction'))
    return render_template('index.html')

@app.route('/prediction')
def prediction():
    

    my_image = dicom.dcmread('temp.dcm')
    
    
    cv2.imwrite('tempfile.png',my_image.pixel_array)
    my_image = cv2.imread('tempfile.png')
    my_image_re = np.array(cv2.resize(my_image,dsize = (256,256))).reshape(-1,256,256,1)
    
    probabilities = model.predict(my_image_re)[0]
    if probabilities[0] <= 0.3:
        predictions = {
            "is_pnemonic":'Normal',
            "probability":probabilities[0],
            "result":""
        }
    
    elif probabilities[0] <= 0.6:
        predictions = {
            "is_pnemonic":'Normal',
            "probability":probabilities[0],
            "result":""
        }
    
    else:
        predictions = {
            "is_pnemonic":'Pneumonic',
            "probability":probabilities[0],
            "result" : ""
        }
   
    return render_template('predict.html', predictions=predictions)

app.run(host='0.0.0.0', port=1000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://10.205.51.105:1000/ (Press CTRL+C to quit)
10.205.51.105 - - [16/Sep/2022 17:35:26] "GET / HTTP/1.1" 200 -
10.205.51.105 - - [16/Sep/2022 17:35:52] "POST / HTTP/1.1" 302 -


1/1 [==============================] - 0s 233ms/step


10.205.51.105 - - [16/Sep/2022 17:35:52] "GET /prediction HTTP/1.1" 200 -
